# Emotion classification Part 1:

# Importing Libraries

In [3]:
import pandas as pd
import numpy as np
 
import nltk
from nltk.corpus import stopwords

from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook.pandas()
from gensim.models import FastText

import contractions  # Assuming you want to use contractions.fix()
import string, re

from nltk.stem import WordNetLemmatizer


# Data cleaning

In [8]:
data = pd.read_csv(r"C:\Users\bobna\Desktop\emotion detection\datasetscrowdflower-sentiment-analysis-in-text\text_emotion.csv")


In [11]:
data.head()

tweet_id   sentiment       author  \
0  1956967341       empty   xoshayzers   
1  1956967666     sadness    wannamama   
2  1956967696     sadness    coolfunky   
3  1956967789  enthusiasm  czareaquino   
4  1956968416     neutral    xkilljoyx   

                                             content  
0  @tiffanylue i know  i was listenin to bad habi...  
1  Layin n bed with a headache  ughhhh...waitin o...  
2                Funeral ceremony...gloomy friday...  
3               wants to hang out with friends SOON!  
4  @dannycastillo We want to trade with someone w...

In [12]:
data.sentiment.value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [13]:
df = data.copy()

In [14]:
df.dtypes

tweet_id      int64
sentiment    object
author       object
content      object
dtype: object

In [15]:
# Dropping rows with other emotion labels, i.e rounding number of emotions to only 5 basic ones 
df = df.drop(df[df.sentiment == 'boredom'].index)
df = df.drop(df[df.sentiment == 'enthusiasm'].index)
df = df.drop(df[df.sentiment == 'empty'].index)
df = df.drop(df[df.sentiment == 'fun'].index)
df = df.drop(df[df.sentiment == 'relief'].index)
df = df.drop(df[df.sentiment == 'surprise'].index)
df = df.drop(df[df.sentiment == 'love'].index)
df = df.drop(df[df.sentiment == 'hate'].index)

In [16]:
df.sentiment.value_counts()

sentiment
neutral      8638
worry        8459
happiness    5209
sadness      5165
anger         110
Name: count, dtype: int64

In [17]:
df

tweet_id  sentiment         author  \
1      1956967666    sadness      wannamama   
2      1956967696    sadness      coolfunky   
4      1956968416    neutral      xkilljoyx   
5      1956968477      worry  xxxPEACHESxxx   
6      1956968487    sadness       ShansBee   
...           ...        ...            ...   
39992  1753918881    neutral    _Alectrona_   
39993  1753918892    neutral     bushidosan   
39994  1753918900  happiness   courtside101   
39995  1753918954    neutral  showMe_Heaven   
39998  1753919043  happiness       ipdaman1   

                                                 content  
1      Layin n bed with a headache  ughhhh...waitin o...  
2                    Funeral ceremony...gloomy friday...  
4      @dannycastillo We want to trade with someone w...  
5      Re-pinging @ghostridah14: why didn't you go to...  
6      I should be sleep, but im not! thinking about ...  
...                                                  ...  
39992  @jasimmo Ooo showing of your French skills!! l...  
39993  @sendsome2me haha, yeah. Twitter has many uses...  
39994                      Succesfully following Tayla!!  
39995                                   @JohnLloydTaylor  
39998  @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...  

[27581 rows x 4 columns]

In [18]:
df = df.drop('author',axis=1)
df

tweet_id  sentiment  \
1      1956967666    sadness   
2      1956967696    sadness   
4      1956968416    neutral   
5      1956968477      worry   
6      1956968487    sadness   
...           ...        ...   
39992  1753918881    neutral   
39993  1753918892    neutral   
39994  1753918900  happiness   
39995  1753918954    neutral   
39998  1753919043  happiness   

                                                 content  
1      Layin n bed with a headache  ughhhh...waitin o...  
2                    Funeral ceremony...gloomy friday...  
4      @dannycastillo We want to trade with someone w...  
5      Re-pinging @ghostridah14: why didn't you go to...  
6      I should be sleep, but im not! thinking about ...  
...                                                  ...  
39992  @jasimmo Ooo showing of your French skills!! l...  
39993  @sendsome2me haha, yeah. Twitter has many uses...  
39994                      Succesfully following Tayla!!  
39995                                   @JohnLloydTaylor  
39998  @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...  

[27581 rows x 3 columns]

In [19]:
df.reset_index(drop=True,inplace=True)

In [20]:
df

tweet_id  sentiment  \
0      1956967666    sadness   
1      1956967696    sadness   
2      1956968416    neutral   
3      1956968477      worry   
4      1956968487    sadness   
...           ...        ...   
27576  1753918881    neutral   
27577  1753918892    neutral   
27578  1753918900  happiness   
27579  1753918954    neutral   
27580  1753919043  happiness   

                                                 content  
0      Layin n bed with a headache  ughhhh...waitin o...  
1                    Funeral ceremony...gloomy friday...  
2      @dannycastillo We want to trade with someone w...  
3      Re-pinging @ghostridah14: why didn't you go to...  
4      I should be sleep, but im not! thinking about ...  
...                                                  ...  
27576  @jasimmo Ooo showing of your French skills!! l...  
27577  @sendsome2me haha, yeah. Twitter has many uses...  
27578                      Succesfully following Tayla!!  
27579                                   @JohnLloydTaylor  
27580  @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...  

[27581 rows x 3 columns]

In [21]:
df.shape

(27581, 3)

In [22]:
df.drop('tweet_id',axis=1,inplace=True)

In [23]:
df.columns = ['sentiment','text']

# Data pre-processing
### Stopword Exclusion

In [24]:
#Using Sentiment lexicons to be excluded from stopwords

In [30]:
df_pos = pd.read_csv(r"C:\Users\bobna\Desktop\emotion detection\lexicons\positive.csv.csv")
df_neg = pd.read_csv(r"C:\Users\bobna\Desktop\emotion detection\lexicons\negative.csv.csv")


# Helper functions

In [31]:
def expand_text(text):
    text = text.lower()
    text = text.replace("`","'")
    
    #Expand Contractions
    contraction_dict = contraction_map
    contraction_keys = list(contraction_dict.keys())
    
    for word in text.split():
        if word in contraction_keys:
            text = text.replace(word, contraction_dict[word])
        else:
            continue
    
    return text    

In [32]:
def clean_text(text):
    text = text.translate(string.punctuation)
    text = text.lower().split()
    
    df_pos_words = list(df_pos.words)
    df_neg_words = list(df_neg.words)
    
    positive = []
    for i in range(0,len(df_pos_words)):
        positive.append(df_pos_words[i].lower().replace(" ",""))
        
    negative = []
    for i in range(0,len(df_neg_words)):
        negative.append(df_neg_words[i].lower().replace(" ",""))
        
    pos_set = set(positive)
    neg_set = set(negative)
    
    keywords = set(["above","and","below","not"])
    
    keywords.update(pos_set)
    keywords.update(neg_set)
    
    stopwords_set = set(stopwords.words('english'))
    stops = stopwords_set - keywords
    
    
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    text = re.sub(r"[^A-Za-z0-9^,!./\'+-=]"," ",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"\'s"," ",text)
    text = re.sub(r"\'ve"," have ",text)
    text = re.sub(r"n't"," not ",text)
    text = re.sub(r"i'm"," i am ",text)
    text = re.sub(r"\'re"," are ",text)
    text = re.sub(r"\'d", " would ",text)
    text = re.sub(r"\'ll", " will ",text)
    text = re.sub(r","," ",text)
    text = re.sub(r"\."," ",text)
    text = re.sub(r"!"," ! ",text)
    text = re.sub(r"\/"," ",text)
    text = re.sub(r"\^"," ^ ",text)
    text = re.sub(r"\+"," + ",text)
    text = re.sub(r"\-"," - ",text)
    text = re.sub(r"\="," = ",text)
    text = re.sub(r"'"," ",text)
    text = re.sub(r"(\d+)(k)",r"\g<1>000",text)
    text = re.sub(r":", " : ",text)
    text = re.sub(r" e g "," eg ",text)
    text = re.sub(r"b g "," bg ",text)
    text = re.sub(r" u s "," american ",text)
    text = re.sub(r"\0s","0",text)
    text = re.sub(r"e - mail","email",text)
    text = re.sub(r"\s{2,}"," ",text)
    
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemmatized_words)
    
    return text

In [33]:
df

sentiment                                               text
0        sadness  Layin n bed with a headache  ughhhh...waitin o...
1        sadness                Funeral ceremony...gloomy friday...
2        neutral  @dannycastillo We want to trade with someone w...
3          worry  Re-pinging @ghostridah14: why didn't you go to...
4        sadness  I should be sleep, but im not! thinking about ...
...          ...                                                ...
27576    neutral  @jasimmo Ooo showing of your French skills!! l...
27577    neutral  @sendsome2me haha, yeah. Twitter has many uses...
27578  happiness                      Succesfully following Tayla!!
27579    neutral                                   @JohnLloydTaylor
27580  happiness  @niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEE...

[27581 rows x 2 columns]

In [35]:
import contractions

def expand_text(text):
    text = text.lower()
    text = contractions.fix(text)
    return text


In [36]:
df['text'] = df['text'].progress_apply(lambda x : expand_text(x))

  0%|          | 0/27581 [00:00<?, ?it/s]

In [37]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bobna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bobna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
df['text'] = df['text'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/27581 [00:00<?, ?it/s]

In [39]:
df.sentiment.value_counts()

sentiment
neutral      8638
worry        8459
happiness    5209
sadness      5165
anger         110
Name: count, dtype: int64

In [40]:
df

sentiment                                               text
0        sadness            layin n bed headache ughhhh waitin call
1        sadness                     funeral ceremony gloomy friday
2        neutral  dannycastillo want trade someone houston ticke...
3          worry  re - pinging ghostridah14 : not go prom bf not...
4        sadness  sleep not ! thinking old friend want married n...
...          ...                                                ...
27576    neutral  jasimmo ooo showing french skill ! ! lol thing...
27577    neutral  sendsome2me haha yeah twitter many us know peo...
27578  happiness                    succesfully following tayla ! !
27579    neutral                                    johnlloydtaylor
27580  happiness  niariley wassup beautiful ! ! ! follow me ! ! ...

[27581 rows x 2 columns]

In [41]:
#Feature encoding
from sklearn.preprocessing import LabelEncoder
lbl_enc = LabelEncoder()

In [42]:
y = lbl_enc.fit_transform(df.sentiment.values) #Label encoding 

In [43]:
#For building a dataframe for mapping emotions to label number
y_series = pd.Series(y)
emo_cols_series = pd.Series(lbl_enc.inverse_transform(y))  #inverse transform to find  mapped emotions

In [44]:
emo_df = pd.DataFrame()
emo_df = pd.concat([emo_cols_series,y_series],axis=1)
emo_df.columns=['emotion','Label_mapped']

In [ ]:
emo_df = emo_df.drop_duplicates()
emo_df.sort_values(by=['Label_mapped'],ascending= True)

# Model building
### Libraries

In [46]:
#Train test split 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train,X_val,y_train,y_val = train_test_split(df.text.values,df.sentiment.values,random_state=42, test_size=0.1, shuffle=True)

# Using Tf-idf

In [47]:
#Extracting Tf-idf features

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

In [49]:
#TF-idf approach
tfidf.fit(list(X_train) + list(X_val))
X_train_tfidf = tfidf.transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

# Using Countvec

In [50]:
#Count Vector approach
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3))
count_vec.fit(list(X_train) + list(X_val))

CountVectorizer(ngram_range=(1, 3), token_pattern='\\w{1,}')

In [51]:
X_train_count_vec = count_vec.transform(X_train)
X_val_count_vec = count_vec.transform(X_val)

# Final Result:

In [52]:
#Using tf-idf for models

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
y_pred = nb.predict(X_val_tfidf)
print('naive bayes tfidf accuracy %s' % accuracy_score(y_pred, y_val))

naive bayes tfidf accuracy 0.4331279449075752


In [57]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train_tfidf,y_train)
y_pred2 = rf.predict(X_val_tfidf)
print('Random forest tfidf accuracy %s' % accuracy_score(y_pred2, y_val))

Random forest tfidf accuracy 0.4555998550199348


In [70]:
from sklearn.linear_model import SGDClassifier
lsvm = SGDClassifier(alpha=0.001, random_state=5, max_iter=15, tol=None)
lsvm.fit(X_train_tfidf, y_train)
y_pred_sgd = lsvm.predict(X_val_tfidf)
print('SGD using tfidf accuracy %s' % accuracy_score(y_pred_sgd, y_val))

SGD using tfidf accuracy 0.4729974628488583


In [71]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)
lr.fit(X_train_tfidf,y_train)
y_pred = lr.predict(X_val_tfidf)
print('Accuracy with Logistic Regression and Tf-idf is {}'.format(accuracy_score(y_pred,y_val)))

Accuracy with Logistic Regression and Tf-idf is 0.4621239579557811


C:\Users\bobna\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [72]:
#Using Countvec for model

from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_count_vec, y_train)
y_pred = nb.predict(X_val_count_vec)
print('naive bayes count_vec accuracy %s' % accuracy_score(y_pred, y_val))

naive bayes count_vec accuracy 0.4385646973541138


In [73]:
from sklearn.linear_model import SGDClassifier
lsvm = SGDClassifier()
lsvm.fit(X_train_count_vec, y_train)
y_pred = lsvm.predict(X_val_count_vec)
print('svm using countvec accuracy %s' % accuracy_score(y_pred, y_val))

svm using countvec accuracy 0.46139905762957595


In [74]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)
lr.fit(X_train_count_vec,y_train)
y_pred = lr.predict(X_val_count_vec)
print('Accuracy with Logistic Regression and CountVec is {}'.format(accuracy_score(y_pred,y_val)))

Accuracy with Logistic Regression and CountVec is 0.4671982602392171


C:\Users\bobna\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Generate classification report and confusion matrix for SGD + TF-IDF model

In [75]:
from sklearn.metrics import classification_report

In [76]:
print(classification_report(y_val,y_pred_sgd))

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00         4
   happiness       0.53      0.57      0.55       532
     neutral       0.44      0.54      0.49       827
     sadness       0.45      0.26      0.33       518
       worry       0.48      0.48      0.48       878

    accuracy                           0.47      2759
   macro avg       0.38      0.37      0.37      2759
weighted avg       0.47      0.47      0.47      2759

